## 빅데이터 실습

### 스타벅스 매장 입지 분석

#### 데이터 수집

##### 셀레니움 사용 자동화

In [1]:
# 필요 라이브러리 사용등록
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# 웹드라이버 스타벅스 웹사이트 오픈
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [19]:
## 웹드라이버에서 소스 위치의 서울 링크를 클릭
name = 'set_sido_cd_btn'
driver.find_elements(By.CLASS_NAME, name)[0].click()

In [4]:
# ## 웹드라이버에서 소스 위치의 부산 링크를 클릭
# name = 'set_sido_cd_btn'
# driver.find_elements(By.CLASS_NAME, name)[5].click()

In [20]:
# 웹드라이버에서 소스 위치 전체 링크 클릭
name = 'set_gugun_cd_btn'
driver.find_elements(By.CLASS_NAME, name)[0].click()

In [21]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [22]:
#검색결과 613개 겟
sbSeoulList = soup.select('li.quickResultLstCon')
len(sbSeoulList)

613

In [23]:
sbSeoulList[8]

<li class="quickResultLstCon" data-code="3648" data-hlytag="null" data-index="8" data-lat="37.507750" data-long="127.060651" data-name="삼성역섬유센터R" data-storecd="1438" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="삼성역섬유센터R" data-store="1438" data-yn="N">삼성역섬유센터R  </strong> <p class="result_details">서울특별시 강남구 테헤란로 518 (대치동)<br/>1522-3232</p> <i class="pin_reserve">리저브 매장 2번</i></li>

##### 서울 매장 리스트 DF

In [26]:
sbSeoulStores = []
for item in sbSeoulList: # 613번 돌면서
    name = item.select('strong')[0].text.strip() #매장명
    lat = item['data-lat'].strip() # 위도
    lng = item['data-long'].strip() # 경도
    strongType = item.select('i')[0]['class'][0].split('_')[1] # 가게 종류(일반,리저브,DT,WT)
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    
    sbSeoulStores.append([name,lat,lng,strongType,address,tel])

In [27]:
#DF로 변경
dfStarBucksSeoul = pd.DataFrame(sbSeoulStores, columns=['매장명', '위도', '경도', '매장타입', '주소', '전화번호'])

In [28]:
dfStarBucksSeoul

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.5139309,127.0206057,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩,37.494668,127.062583,general,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
...,...,...,...,...,...,...
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232
609,상봉역,37.59689,127.08647,general,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232
611,양원역,37.6066536267232,127.106359790053,general,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232


In [29]:
# 결측치가 하나도 없음
dfStarBucksSeoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613 entries, 0 to 612
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     613 non-null    object
 1   위도      613 non-null    object
 2   경도      613 non-null    object
 3   매장타입    613 non-null    object
 4   주소      613 non-null    object
 5   전화번호    613 non-null    object
dtypes: object(6)
memory usage: 28.9+ KB


In [32]:
dfStarBucksSeoul.to_excel('./data/startbucks_seoul_list.xlsx', index=False)

##### 서울열린데이터광장 OpenAPI 공공데이터 수집

- 아래방법 동작안함

In [33]:
#서울열린데이터광장 API 키
SEOUL_API_KEY = '425a6a51786b61723131397852756269'

In [34]:
# 서울열린데이터광장 호출 URL
url = f'http://openapi.seoul.go.kr:8088/{SEOUL_API_KEY}/json/SdeTlSccoSigW/1/25'
url

'http://openapi.seoul.go.kr:8088/425a6a51786b61723131397852756269/json/SdeTlSccoSigW/1/25'

##### 행정구역 데이터(위경도 + 구역)
- https://github.com/southkorea/seoul-maps/blob/master/kostat/2013/json/seoul_municipalities_geo_simple.json